In [1]:
from multitask_dataset import SingerMultiTaskDataset
from torch.utils.data import DataLoader
from models import HuBERTMultiHead
import pandas as pd

In [2]:
train_audio_folder = '/media/maximos/9C33-6BBD/data/melos_singers/Rebetika_vowels/train/'
csv_path = '/media/maximos/9C33-6BBD/data/melos_singers/features/multitask_targets.csv'

In [3]:
feats = pd.read_csv(csv_path, delimiter=',')
features_list = list(feats.columns)
print(features_list)
del(features_list[:2])
print(features_list)

['Unnamed: 0', 'names', 'singer_id', 'Pitch', 'SpectralCentroid', 'SpectralSpread', 'SpectralSkewness', 'SpectralKurtosis', 'SpectralFlatness', 'SpectralCrest', 'SpectralSlope', 'SpectralDecrease', 'SpectralRollOff', 'SpectralVariation', 'SpectralFlux', 'HarmonicSpectralDeviation', 'Tristimulus_1', 'Tristimulus_2', 'Tristimulus_3', 'HarmonicOddToEvenRatio', 'Inharmonicity', 'HarmonicEnergy', 'NoiseEnergy', 'Noisiness', 'HarmonicToNoiseEnergy', 'PartialsToNoiseEnergy', 'F1_Hz', 'F2_Hz', 'F3_HZ', 'F4_Hz', 'Rate', 'Depth', 'Regularity']
['singer_id', 'Pitch', 'SpectralCentroid', 'SpectralSpread', 'SpectralSkewness', 'SpectralKurtosis', 'SpectralFlatness', 'SpectralCrest', 'SpectralSlope', 'SpectralDecrease', 'SpectralRollOff', 'SpectralVariation', 'SpectralFlux', 'HarmonicSpectralDeviation', 'Tristimulus_1', 'Tristimulus_2', 'Tristimulus_3', 'HarmonicOddToEvenRatio', 'Inharmonicity', 'HarmonicEnergy', 'NoiseEnergy', 'Noisiness', 'HarmonicToNoiseEnergy', 'PartialsToNoiseEnergy', 'F1_Hz', '

In [4]:
task_labels_num_out = {}
for i in range(1, len(features_list)-3, 1):
    task_labels_num_out[features_list[i]] = 1
# add singer identification
task_labels_num_out['singer_id'] = feats['singer_id'].max()+1 # accounting for zero
print(task_labels_num_out)

{'Pitch': 1, 'SpectralCentroid': 1, 'SpectralSpread': 1, 'SpectralSkewness': 1, 'SpectralKurtosis': 1, 'SpectralFlatness': 1, 'SpectralCrest': 1, 'SpectralSlope': 1, 'SpectralDecrease': 1, 'SpectralRollOff': 1, 'SpectralVariation': 1, 'SpectralFlux': 1, 'HarmonicSpectralDeviation': 1, 'Tristimulus_1': 1, 'Tristimulus_2': 1, 'Tristimulus_3': 1, 'HarmonicOddToEvenRatio': 1, 'Inharmonicity': 1, 'HarmonicEnergy': 1, 'NoiseEnergy': 1, 'Noisiness': 1, 'HarmonicToNoiseEnergy': 1, 'PartialsToNoiseEnergy': 1, 'F1_Hz': 1, 'F2_Hz': 1, 'F3_HZ': 1, 'F4_Hz': 1, 'singer_id': 6}


In [5]:
model = HuBERTMultiHead(task_labels_num_out=task_labels_num_out)

You are using a model of type hubert to instantiate a model of type wav2vec2. This is not supported for all configurations of models and can yield errors.
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
training_data = SingerMultiTaskDataset(train_audio_folder, csv_path)

In [7]:
print(len(training_data))

336


In [8]:
print(training_data[235])

{'input_values': array([ 9.6231786e-05, -4.8065151e-04, -1.0562916e-03, ...,
        2.1723036e-02,  3.0537494e-02,  3.0633982e-02], dtype=float32), 'labels': {'Unnamed: 0': 235, 'names': 'keti_grei_e_7', 'singer_id': 3, 'Pitch': 442.967567138073, 'SpectralCentroid': 1649.18342861135, 'SpectralSpread': 1091.55349106278, 'SpectralSkewness': 1.37180571109231, 'SpectralKurtosis': 5.05492277926304, 'SpectralFlatness': 1.19733652424461e-09, 'SpectralCrest': 0.246353359792086, 'SpectralSlope': 1.0414362293082e-06, 'SpectralDecrease': 0.0087732118944919, 'SpectralRollOff': 2131.34777604787, 'SpectralVariation': 0.861851957216352, 'SpectralFlux': 0.001313200733224, 'HarmonicSpectralDeviation': 0.012296689893735, 'Tristimulus_1': 0.250974023096942, 'Tristimulus_2': 0.590761277109722, 'Tristimulus_3': 0.140094494653077, 'HarmonicOddToEvenRatio': 0.381924585198844, 'Inharmonicity': 0.0848189851447228, 'HarmonicEnergy': 0.0184961146711919, 'NoiseEnergy': 0.029466693442144, 'Noisiness': 0.498319194

In [9]:
dataloader = DataLoader(training_data, batch_size=8, shuffle=True, collate_fn=model.collate_fn)

In [10]:
print(dataloader.__dict__)

{'dataset': <multitask_dataset.SingerMultiTaskDataset object at 0x770f70117a70>, 'num_workers': 0, 'prefetch_factor': None, 'pin_memory': False, 'pin_memory_device': '', 'timeout': 0, 'worker_init_fn': None, '_DataLoader__multiprocessing_context': None, '_dataset_kind': 0, 'batch_size': 8, 'drop_last': False, 'sampler': <torch.utils.data.sampler.RandomSampler object at 0x770efbe2e630>, 'batch_sampler': <torch.utils.data.sampler.BatchSampler object at 0x770efbe2d0a0>, 'generator': None, 'collate_fn': <bound method HuBERTMultiHead.collate_fn of HuBERTMultiHead(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
      

In [11]:
b = next(iter(dataloader))

In [13]:
print(len(b[0]['input_values']))
print(len(b[0]['attention_mask']))
print(len(b[1]))
print(b[1])

8
8
33
{'Unnamed: 0': [210, 56, 108, 276, 87, 146, 203, 79], 'names': ['keti_grei_a_1', 'bithikotsis_i_8', 'kazantzidis_i_14', 'ninou_e_4', 'kazantzidis_a_5', 'kazantzidis_old_i_10', 'kazantzidis_w_3', 'kazantzidis_a_18'], 'singer_id': [3, 1, 2, 4, 2, 2, 2, 2], 'Pitch': [535.988137952919, 196.750559613665, 196.296494053867, 440.92541806362, 197.319618870527, 215.925616065187, 267.066879934111, 226.922721646777], 'SpectralCentroid': [1604.22246900928, 1419.84821259399, 1705.6862093644, 2315.1939196014, 1155.50786599725, 1279.92643832382, 1034.77067769143, 1378.59343963769], 'SpectralSpread': [891.23944864804, 1202.39892206693, 1222.06835521984, 980.670723392669, 660.360208233537, 1090.27664505363, 756.039230893439, 789.04951998153], 'SpectralSkewness': [1.30524323043544, 0.699908105151804, 0.263777843232345, -0.423891811380255, 1.63228118840777, 0.815500213748112, 2.20245924197386, 1.19002708161243], 'SpectralKurtosis': [4.184987924076, 2.39864388048342, 2.33527538222561, 2.494295401345

In [14]:
y = model(
    audio_normalized=b[0]['input_values'],
    attention_mask=b[0]['attention_mask'],
    labels=b[1],
    output_attentions=False,
    output_hidden_states=False,
    return_dict=True
)

AttributeError: 'list' object has no attribute 'dtype'